#Computação Quântica - Lista de Exercícios

##Unidade 2

Aluno: Gustavo Pereira de Carvalho

Docente: Anderson Paiva Cruz


In [1]:
#Sempre executar antes das outras células
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 71.7 MB/s eta 0:00:00


1. A partir da plataforma de computação quântica de sua preferência, implemente os seguintes algoritmos quânticos. Ao final de cada algoritmo, faça um relatório apresentando o código e explicando o funcionamento do mesmo.

a) Faça um circuito quântico cujo estado final é |00000❭+|11111❭.

In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Criando 5 qubits
n_qubits = 5
qc = QuantumCircuit(n_qubits)

# Aplicando Hadamard no primeiro qubit
qc.h(0)

# Depois CNOTs entre o primeiro qubit e todos os outros
for target_qubit in range(1, n_qubits):
    qc.cx(0, target_qubit)

print("Visualização do Circuito Quântico")
print(qc.draw(output='text'))

print("\nSimulando o Vetor de Estado")
# Criando o simulador e salvando o estado final
sim = AerSimulator(method='statevector')
qc.save_statevector()

# Transpila e executa
qc_t = transpile(qc, sim)
job = sim.run(qc_t)
result = job.result()

# Salvando com numpy pra evitar warning
statevector = np.asarray(result.data(0)['statevector'])

print("Verificando as amplitudes não nulas do vetor de estado:")
for i, amplitude in enumerate(statevector):
    if not np.isclose(amplitude, 0):
        binary_state = f"{i:0{n_qubits}b}"
        print(f"  |{binary_state}⟩: {amplitude:.5f}")

#   1/sqrt(2) = 0.70711

print("\nSimulando a Medição")
qc_measure = QuantumCircuit(n_qubits, n_qubits)
qc_measure.h(0)
for target_qubit in range(1, n_qubits):
    qc_measure.cx(0, target_qubit)
qc_measure.barrier()
qc_measure.measure(range(n_qubits), range(n_qubits))

# Simulador de medições
qasm_sim = AerSimulator()
qc_measure_t = transpile(qc_measure, qasm_sim)

job_m = qasm_sim.run(qc_measure_t, shots=1024)
result_m = job_m.result()
counts = result_m.get_counts()

print("Resultados da medição (1024 shots):")
print(counts)


Visualização do Circuito Quântico
     ┌───┐                    
q_0: ┤ H ├──■────■────■────■──
     └───┘┌─┴─┐  │    │    │  
q_1: ─────┤ X ├──┼────┼────┼──
          └───┘┌─┴─┐  │    │  
q_2: ──────────┤ X ├──┼────┼──
               └───┘┌─┴─┐  │  
q_3: ───────────────┤ X ├──┼──
                    └───┘┌─┴─┐
q_4: ────────────────────┤ X ├
                         └───┘

Simulando o Vetor de Estado
Verificando as amplitudes não nulas do vetor de estado:
  |00000⟩: 0.70711+0.00000j
  |11111⟩: 0.70711+0.00000j

Simulando a Medição
Resultados da medição (1024 shots):
{'11111': 500, '00000': 524}


A questão pede a criação de um estado $\frac{1}{\sqrt{2}}(|00000\rangle + |11111\rangle)$, que seria um estado em que 5 qubits estão emaranhados, ou todos são 0 ou todos são 1.

Para criar esse estado primeiro é necessário começar com todos os qubits em 0, aplicando uma porta Hadamard ao primeiro qubit, o colocando no estado de superposição.

Depois, basta usar o qubit que está em superposição como controle para aplicar portas CNOT em todos os outros qubits, gerando o emaranhamento e a saida esperada.

b) Implemente a subrotina de teleporte de informação quântica.


In [3]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram

# Estado que vai ser teleportado
theta = np.pi / 3  # Um ângulo qualquer

# Declaração dos registradores e construção do circuito
q = QuantumRegister(3, 'q')
c_alice = ClassicalRegister(2, 'c_alice')
c_bob = ClassicalRegister(1, 'c_bob')

qc = QuantumCircuit(q, c_alice, c_bob)

# Aplicando o estado theta no primeiro qubit de Alice
qc.ry(theta, q[0])
qc.barrier()

# Criação do par entrelaçado entre Alice e Bob
qc.h(q[1])
qc.cx(q[1], q[2])
qc.barrier()

# Medição de Bell (Alice)
qc.cx(q[0], q[1])
qc.h(q[0])
qc.barrier()

# Comunicação clássica (Alice -> Bob)
qc.measure(q[0], c_alice[0]) # q0 -> c_alice[0]
qc.measure(q[1], c_alice[1]) # q1 -> c_alice[1]
qc.barrier()

# Correção (Bob)

with qc.if_test((c_alice[1], 1)): # Se c_alice[1] (de q1) == 1
    qc.x(q[2]) # Aplica X em q2

with qc.if_test((c_alice[0], 1)): # Se c_alice[0] (de q0) == 1
    qc.z(q[2]) # Aplica Z em q2

qc.barrier()

# Verificação de Bob
"""
Aplica -theta para que Bob volte pra 0. Se Bob realmente recebeu theta,
todas as saidas de Bob serão 0, pra qualquer valor de q0 e q1 de Alice.
"""
qc.ry(-theta, q[2])
qc.measure(q[2], c_bob[0]) # q2 -> c_bob[0]

# Visualização do circuito e simulação
print("Circuito de teleporte quântico")
print(qc.draw(output='text'))

# Execução
simulator = AerSimulator()
job = simulator.run(qc, shots=1024)
result = job.result()
counts = result.get_counts()

print("\nResultados da simulação (1024 shots)")
print(counts)

Circuito de teleporte quântico
           ┌─────────┐ ░            ░      ┌───┐ ░ ┌─┐    ░                  »
      q_0: ┤ Ry(π/3) ├─░────────────░───■──┤ H ├─░─┤M├────░──────────────────»
           └─────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░                  »
      q_1: ────────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░──────────────────»
                       ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░      ┌──────     »
      q_2: ────────────░──────┤ X ├─░────────────░──╫──╫──░──────┤ If-0  ────»
                       ░      └───┘ ░            ░  ║  ║  ░      └──╥───     »
                                                    ║  ║    ┌───────╨───────┐»
c_alice: 2/═════════════════════════════════════════╩══╩════╡ c_alice_1=0x1 ╞»
                                                    0  1    └───────────────┘»
  c_bob: 1/══════════════════════════════════════════════════════════════════»
                                                                             »
«                    

O teleporte quântico consiste em transportar um estado quântico, informação nesse caso, de um local para outro. Geralmente o local fonte é chamado de Alice e o destino de Bob.

Para implementação do teleporte são necessários, 1 qubit de Alice que terá o estado que vai ser teleportado, outro qubit de Alice e 1 de Bob que estarão emaranhados e 2 bits clássicos que Alice envia para Bob.

Primeiro um theta qualquer que será enviado é aplicado no qubit 0 de Alice. Depois o qubit 1 de Alice e qubit de Bob são emaranhados, depois é aplicado um conjunto de CNOT e Hadamard nos qubits de Alice para realizar a medição. Por fim os dois bits clássicos de Alice são enviados para Bob e Bob aplica as portas X e/ou Z dependendo dos bits clássicos recebidos.

Para verificar o -theta é aplicado ao qubit de Bob. Se ele estava em theta, voltará para 0, dessa forma é possivel medir seu estado e verificar se teleporte deu certo.

c) Implemente a subrotina de soma completa de dois qubits.


In [4]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile

def create_full_adder_circuit():
    # São 4 qubits no total
    qc = QuantumCircuit(4, name="Full Adder")

    # C_out = (A AND B)
    qc.ccx(0, 1, 3)

    # q[1] = A XOR B
    qc.cx(0, 1)

    # C_out = C_out XOR ( (A XOR B) AND C_in )
    qc.ccx(1, 2, 3)

    # Soma = (A XOR B) XOR C_in
    qc.cx(1, 2)

    return qc

# Testando todas as 8 combinações de entrada (A, B, C_in)
for a in [0, 1]:
    for b in [0, 1]:
        for c_in in [0, 1]:

            # 4 qubits para o circuito, 2 bits clássicos para as saídas
            qc_test = QuantumCircuit(4, 2)

            # Define os estados de entrada A, B, C_in
            if a == 1:
                qc_test.x(0)
            if b == 1:
                qc_test.x(1)
            if c_in == 1:
                qc_test.x(2)
            # q[3] continua em |0>

            qc_test.barrier()

            adder_subroutine = create_full_adder_circuit()
            # Converte a subrotina para "gate" para visualização
            qc_test.append(adder_subroutine.to_gate(), [0, 1, 2, 3])

            qc_test.barrier()

            # Medição das saídas
            # Mede q[2] (Soma) -> bit clássico 0
            # Mede q[3] (C_out) -> bit clássico 1
            qc_test.measure(2, 0) # q_soma para c_soma
            qc_test.measure(3, 1) # q_cout para c_cout

            # Simulação
            simulator = AerSimulator()
            transpiled_qc = transpile(qc_test, simulator)
            job = simulator.run(transpiled_qc, shots=1024) #Só precisaria de 1 shot, mas queria testar com 1024
            result = job.result()
            counts = result.get_counts()

            # Saida no formato  'c_cout c_soma'
            output = list(counts.keys())[0]

            # Soma clássica para comparação
            s_classical = a ^ b ^ c_in
            cout_classical = (a & b) | (c_in & (a ^ b))

            print(f"Entrada (A,B,C_in): ({a},{b},{c_in})")
            print(f"Saída Quântica (C_out, Soma): ({output[0]}, {output[1]})")
            print(f"Saída Clássica:               ({cout_classical}, {s_classical}) \n")

# Visualização do circuito
print("\nCircuito Base do Full Adder")
qc_base = create_full_adder_circuit()
print(qc_base.draw(output='text'))

Entrada (A,B,C_in): (0,0,0)
Saída Quântica (C_out, Soma): (0, 0)
Saída Clássica:               (0, 0) 

Entrada (A,B,C_in): (0,0,1)
Saída Quântica (C_out, Soma): (0, 1)
Saída Clássica:               (0, 1) 

Entrada (A,B,C_in): (0,1,0)
Saída Quântica (C_out, Soma): (0, 1)
Saída Clássica:               (0, 1) 

Entrada (A,B,C_in): (0,1,1)
Saída Quântica (C_out, Soma): (1, 0)
Saída Clássica:               (1, 0) 

Entrada (A,B,C_in): (1,0,0)
Saída Quântica (C_out, Soma): (0, 1)
Saída Clássica:               (0, 1) 

Entrada (A,B,C_in): (1,0,1)
Saída Quântica (C_out, Soma): (1, 0)
Saída Clássica:               (1, 0) 

Entrada (A,B,C_in): (1,1,0)
Saída Quântica (C_out, Soma): (1, 0)
Saída Clássica:               (1, 0) 

Entrada (A,B,C_in): (1,1,1)
Saída Quântica (C_out, Soma): (1, 1)
Saída Clássica:               (1, 1) 


Circuito Base do Full Adder
                         
q_0: ──■────■────────────
       │  ┌─┴─┐          
q_1: ──■──┤ X ├──■────■──
       │  └───┘  │  ┌─┴─┐
q_2: ──┼─

O somador completo soma 3 bits, A, B e C_in, dando como saida a soma e um C_out. Nesse caso, o código usa C_in = 0 para somar apenas A e B, sendo assim, temos 3 qubits mais um qubit ancilla para o C_out. A soma (A ⊕ B ⊕ C_in) será armazenada no qubit q2 e o C_out ((A·B) + (C_in·(A⊕B))) no q3.

O circuito foi feito utilizando portas CNOT e CCX:

CCX(q0, q1, q3): Calcula A·B e armazena em q3
  q3 agora é (A·B).

CNOT(q0, q1): Calcula A⊕B e armazena em q1
  q1 agora é (A⊕B).

CCX(q1, q2, q3): Calcula (A⊕B)·C_in e faz um XOR com q3
  q3 agora é (A·B) ⊕ ((A⊕B)·C_in), a fórmula booleana para o C_out

CNOT(q1, q2): Calcula (A⊕B) ⊕ C_in e armazena em q2
  q2 agora é (A⊕B) ⊕ C_in, a fórmula para a soma

No fim q2 contém a soma e q3 o C_out.

d) Faça o algoritmo de Deutsch-Josza com 3 qubits de entrada e f(x)=x0⊕x1x2.


In [5]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram

n = 3 # 3 qubits de entrada

# 3 qubits + 1 qubit ancilla
# O ancilla terá indice n
qc = QuantumCircuit(n + 1, n)

# Coloca a ancilla no estado |1>
qc.x(n)

# Aplica Hadamard em todo os qubits
qc.h(range(n + 1))
qc.barrier()

# f(x) = x0 ⊕ (x1 * x2)

qc.cx(0, n) # CNOT(q0, ancilla)

# Porta Toffoli: (q1 é controle, q2 é controle, n é alvo)
qc.ccx(1, 2, n)

qc.barrier()

# Aplica Hadamard nos qubits de entrada (de 0 a n-1)
qc.h(range(n))
qc.barrier()

# Mede os qubits de entrada
qc.measure(range(n), range(n))

# Visualização
print("Circuito de Deutsch-Jozsa (n=3)")
print("f(x) = x0 ⊕ x1 x2 (Balanceada)")
print(qc.draw(output='text'))

# Simulação
simulator = AerSimulator()
transpiled_qc = transpile(qc, simulator)
job = simulator.run(transpiled_qc, shots=1024)
result = job.result()
counts = result.get_counts()

print("\nResultados da medição")
print(counts)

Circuito de Deutsch-Jozsa (n=3)
f(x) = x0 ⊕ x1 x2 (Balanceada)
     ┌───┐      ░            ░ ┌───┐ ░ ┌─┐      
q_0: ┤ H ├──────░───■────────░─┤ H ├─░─┤M├──────
     ├───┤      ░   │        ░ ├───┤ ░ └╥┘┌─┐   
q_1: ┤ H ├──────░───┼────■───░─┤ H ├─░──╫─┤M├───
     ├───┤      ░   │    │   ░ ├───┤ ░  ║ └╥┘┌─┐
q_2: ┤ H ├──────░───┼────■───░─┤ H ├─░──╫──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌─┴─┐ ░ └───┘ ░  ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────░──╫──╫──╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ░  ║  ║  ║ 
c: 3/═══════════════════════════════════╩══╩══╩═
                                        0  1  2 

Resultados da medição
{'101': 260, '001': 272, '111': 236, '011': 256}


O algoritmo de Deutsch-Jozsa é usado para determinar se uma função é constante (retorna 0 ou 1 para todas as entradas) ou balanceada (retorna 0 para exatamente metade das entradas e 1 para a outra metade). Nesse caso temos 3 qubits de entrada e precisamos de 1 qubit ancilla.

Verificando manualmente a função f(x)=x0⊕x1x2, vemos que ela é balanceada. Logo, os resultados da medição devem ser diferentes de 000.

O primeiro passo no circuito é aplicar Hadamard em todos os qubits de entrada, dessa forma o oraculo pode calcular f(x) para todos os qubits de entrada de uma vez. Após isso é aplicado uma porta X seguida de Hadamard no oráculo, o deixando no estado  $|-\rangle$. Quando o qubit ancilla está no estado $|-\rangle$, um CNOT (ou qualquer porta controlada) não "flipa" o alvo. Em vez disso, ele aplica uma fase de -1 ao qubit de controle se o controle for $|1\rangle$. Então f(x) é aplicado.

Por fim é aplicada uma segunda camada de portas Hadamard em todos os qubits de entrada fazendo com que os 8 caminhos quânticos interfiram uns nos outros. Se a função fosse Constante, todas as fases seriam iguais e interferência seria 100% construtiva no estado $|000\rangle$. Se a função for Balanceada, as fases positivas e negativas se cancelam perfeitamente (interferência destrutiva) no estado $|000\rangle$, resultando em 0% de probabilidade de medi-lo.

e) Um algoritmo que resolve o problema de Bernstein-Vazirani.


In [6]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram

# Definindo a string secreta
secret_string = '10110'
n = len(secret_string)

# n qubits de entrada + 1 qubit ancilla
qc = QuantumCircuit(n + 1, n)

# Coloca a ancilla no estado |1>
qc.x(n)

# Aplica Hadamard em todos os qubits (incluindo o ancilla)
qc.h(range(n + 1))
qc.barrier()

"""
f(x) = s . x
CNOT(q_i, ancilla) se s_i == 1
Qiskit ordena os bits em little endian
É necessário reverter a string para mapear s_0 -> q_0, s_1 -> q_1...
"""
s_reversed = secret_string[::-1] # Torna-se '01101'

for i, bit in enumerate(s_reversed):
    if bit == '1':
        print(f"Aplicando CNOT de q_{i} (s_{i}) para a ancilla.")
        qc.cx(i, n) # i é o qubit de controle, n é a ancilla

qc.barrier()

# Aplica Hadamard nos qubits de ENTRADA (0 a n-1)
qc.h(range(n))
qc.barrier()

# Mede os qubits de entrada
qc.measure(range(n), range(n))

# Visualização
print(f"\nCircuito de Bernstein-Vazirani (n={n})")
print(f"A string secreta é: {secret_string}")
print(qc.draw(output='text'))

# Simulação
simulator = AerSimulator()
transpiled_qc = transpile(qc, simulator)
job = simulator.run(transpiled_qc, shots=1024) # 1 shot é suficiente, 1024 é só pra teste
result = job.result()
counts = result.get_counts()

print("\nResultado da Medição (1024 shot)")
print(counts)

measured_string = list(counts.keys())[0]
print(f"\nString medida: {measured_string}")
if measured_string == secret_string:
    print("A string secreta foi encontrada.")
else:
    print("Falha.")

Aplicando CNOT de q_1 (s_1) para a ancilla.
Aplicando CNOT de q_2 (s_2) para a ancilla.
Aplicando CNOT de q_4 (s_4) para a ancilla.

Circuito de Bernstein-Vazirani (n=5)
A string secreta é: 10110
     ┌───┐      ░                 ░ ┌───┐ ░ ┌─┐            
q_0: ┤ H ├──────░─────────────────░─┤ H ├─░─┤M├────────────
     ├───┤      ░                 ░ ├───┤ ░ └╥┘┌─┐         
q_1: ┤ H ├──────░───■─────────────░─┤ H ├─░──╫─┤M├─────────
     ├───┤      ░   │             ░ ├───┤ ░  ║ └╥┘┌─┐      
q_2: ┤ H ├──────░───┼────■────────░─┤ H ├─░──╫──╫─┤M├──────
     ├───┤      ░   │    │        ░ ├───┤ ░  ║  ║ └╥┘┌─┐   
q_3: ┤ H ├──────░───┼────┼────────░─┤ H ├─░──╫──╫──╫─┤M├───
     ├───┤      ░   │    │        ░ ├───┤ ░  ║  ║  ║ └╥┘┌─┐
q_4: ┤ H ├──────░───┼────┼────■───░─┤ H ├─░──╫──╫──╫──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌─┴─┐┌─┴─┐ ░ └───┘ ░  ║  ║  ║  ║ └╥┘
q_5: ┤ X ├┤ H ├─░─┤ X ├┤ X ├┤ X ├─░───────░──╫──╫──╫──╫──╫─
     └───┘└───┘ ░ └───┘└───┘└───┘ ░       ░  ║  ║  ║  ║  ║ 
c: 5/═══════════════════

O algoritmo de Bernstein-Vazirani é bem parecido com o de Deutsch-Jozsa, porém ele resolve o problema de uma string secreta. No caso, o oráculo esconde uma string de n bits e uma entrada x é aceita pelo circuito, calculando o produto escalar (mod 2) para entre x e a string secreta para descobr-lá. A vantagem do algoritmo é que no caso clássico seriam necessárias n consultas, enquanto no caso quântico a string inteira é encontrada com 1 consulta.

Assim como no Deutsch-Jozsa o circuito inicia aplicando Hadamard em todos os qubits de entrada e o ancilla é colocado no estado  $|-\rangle$. Após isso, um loop for aplica CNOT apenas se o qubit de entrada for 1, construindo o oráculo para a string secreta. Por fim, mais uma camada de Hadamard é aplicada no qubits de entrada.

Outro ponto importante é o algoritmo de Bernstein-Vazirani é deterministico, então apenas 1 shot seria suficiente para obter o resultado.

f) O algoritmo de Grover para 2 qubits e o valor em |β❭=01.

In [7]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram

# Funções Auxiliares
def create_oracle(qc, qubits_to_mark):
    #Aplica o oráculo para marcar o estado |01>
    qc.x(qubits_to_mark[1])          # inverter q1
    qc.cz(qubits_to_mark[0], qubits_to_mark[1])
    qc.x(qubits_to_mark[1])          # desfazer inversão

def create_diffuser(qc, qubits):
    #Aplica o difusor de Grover (inversão sobre a média)
    qc.h(qubits)
    qc.x(qubits)
    qc.cz(qubits[0], qubits[1])
    qc.x(qubits)
    qc.h(qubits)

# Criando os 2 qubits
n = 2
qubits = list(range(n))

# Simulação (Vetor de Estado)
print("Simulação do Vetor de Estado")

qc_sv = QuantumCircuit(n)
qc_sv.h(qubits)
qc_sv.barrier()
create_oracle(qc_sv, qubits)
qc_sv.barrier()
create_diffuser(qc_sv, qubits)
qc_sv.barrier()

# Usando Statevector diretamente (AerSimulator gera um erro)
statevector = Statevector.from_instruction(qc_sv)

print("Amplitudes finais para |00>, |01>, |10>, |11>:")
print(np.round(statevector.data, 3))

# Medição
print("\nSimulação de Medição")

qc_measure = QuantumCircuit(n, n)
qc_measure.h(qubits)
qc_measure.barrier()
create_oracle(qc_measure, qubits)
qc_measure.barrier()
create_diffuser(qc_measure, qubits)
qc_measure.barrier()
qc_measure.measure(qubits, qubits)

print("\nCircuito final do algoritmo de Grover (n=2, marca='01')")
print(qc_measure.draw(output='text'))

qasm_sim = AerSimulator()
transpiled_qc_m = transpile(qc_measure, qasm_sim)
job_m = qasm_sim.run(transpiled_qc_m, shots=1024)
result_m = job_m.result()
counts = result_m.get_counts()

print("\nResultados da Medição (1024 shots)")
print(counts)


Simulação do Vetor de Estado
Amplitudes finais para |00>, |01>, |10>, |11>:
[-0.+0.j -1.+0.j -0.+0.j -0.+0.j]

Simulação de Medição

Circuito final do algoritmo de Grover (n=2, marca='01')
     ┌───┐ ░               ░ ┌───┐┌───┐   ┌───┐┌───┐ ░ ┌─┐   
q_0: ┤ H ├─░───────■───────░─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░─┤M├───
     ├───┤ ░ ┌───┐ │ ┌───┐ ░ ├───┤├───┤ │ ├───┤├───┤ ░ └╥┘┌─┐
q_1: ┤ H ├─░─┤ X ├─■─┤ X ├─░─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░──╫─┤M├
     └───┘ ░ └───┘   └───┘ ░ └───┘└───┘   └───┘└───┘ ░  ║ └╥┘
c: 2/═══════════════════════════════════════════════════╩══╩═
                                                        0  1 

Resultados da Medição (1024 shots)
{'01': 1024}


O algoritmo de Grover é um algoritmo de busca quântica, o objetivo nesse caso é encontrar um item marcado dentro de um "banco de dados". Como são apenas 2 qubits, temos 4 items possiveis e o item marcado é |β❭=01. A vantagem de usar Grover é sua complexidade $O(\sqrt{N})$, que permite encontrar o item mais rápido que uma busca clássica.

Algumas funções auxiliares são usada no código, a primeira é uma função para criar o oráculo, marcando o estado alvo aplicando uma inversão de fase:

qc.x(q[1]): "Flipa" o estado (ex: $|01\rangle$ para $|11\rangle$).

qc.cz(q[0], q[1]): A porta Controlada-Z aplica a fase de -1 apenas ao estado $|11\rangle$.

qc.x(q[1]): "Desflipa" o estado de volta ($-|11\rangle$ para $-|01\rangle$).

Dessa forma o estado $|01\rangle$ torna-se $-|01\rangle$, e todos os outros estados ($|00\rangle, |10\rangle, |11\rangle$) permanecem inalterados. Marcando o $|01\rangle$.

A segunda função auxiliar é o difusor, que tem a função de amplificar a amplitude do item marcado, que está negativo e diminuir a amplitude dos outros. Isso é feito com duas camadas de Hadamard.

Por fim, a execução principal consiste na simulação do vetor de estados, colocando os qubits em superposição e aplicando as funções auxiliares, para um resultado mais teórico.

Após isso é simulada a medição para se apróximar do que um computador quântico faria, com a mesma sequência, aplicando Hadamard e depois as funções auxiliares, temos o resultado das 1024 execuções. Que corresponde ao alvo esperado, que teve sua amplitude amplificada.